수치 미분은 오래 걸림. 근본적인 무리가 있음
매개변수의 최적값을 찾는 것이 학습
출력층에서 입력층 방향으로 오차를 전파시키며 가중치를 업데이트
수치미분보다 계산 비용 낮고 오차도 적음

역전파 할때는 순전파의 값이 있어야 함
순전파를 하고 나서 역전파를 해야함.
역전파를 통해 미분을 효율적으로 계산할 수 있음.

In [ ]:
# 단순한 계층 구현하기
# 곱셈 노드 MulLayer 덧셈 노드 AddLayer 순전파 forward() 역전파 backward()
# dout는 상류에서 넘어본 미분값.

class MulLayer:
    # __init__()에서는 인스턴스 변수인 x와 y를 초기화한다.
    # 사용 이유: 순전파 시 입력값을 유지하기 위해 사용.
    # 값을 유지해둬야 역전파에서 사용할 수 있기 때문.
    def __init__(self):
        self.x = None
        self.y = None

    # 곱셈 순전파는 곱함
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    # 곱셈 역전파는 서로 바꾼 미분값을 곱함
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

class AddLayer:
    # 곱셈과 달리 덧셈은 입력값을 유지할 필요가 없다. 오염될 일이 없기 때문!
    def __init__(self):
        pass
    # 순전파에서는 그냥 더하면 됨.
    def forward(self, x, y):
        out = x + y
        return out 

    # 역전파에서는 상류에서 내려온 미분을 그대로 하류로 보내면 됨.
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy



# forward(), backward() 테스트
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 220

#역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax) # 2.2 110 200

#-------------------------------------------------------------------

# 덧셈 계층 테스트
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print(price) # 715

#역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dorange, dorange_num, dtax) # 2.2 110 3.3 165 650



활성화 함수 : 입력 신호의 총합을 출력 신호로 변환
ReLU 함수 : y = x (x>0)
            y = 0 (x<=0)
미분한 ReLU 값 : y = 1 (x>0)
                 y = 0 (x<=0)

ReLU 클래스는 mask라는 인스턴스 변수를 가진다.
mask는 True/False로 구성된 넘파이 배열로 x의 원소값이 0이하인 인덱스는 True,
0보다 큰 원소는 False를 유지한다.
순전파 때의 입력값이 0 이하이면 역전파 때의 값은 0이 돼야 한다.
그래서 역전파 때는 순전파 때 만들어둔 mask를 써서 mask의 원소가 True인 곳에는 상류에서
전파된 dout(미분값)을 0으로 설정한다.

